In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import cv2
import os

def extract_frames(video_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_dir, f"frame_{count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1
    cap.release()

# Extract frames from train and test videos
extract_frames('/kaggle/input/q3-computer-vision/train.mp4', '/kaggle/working/train_frames')
extract_frames('/kaggle/input/q3-computer-vision/test.mp4', '/kaggle/working/test_frames')


In [4]:
import numpy as np
import os
import cv2

def calculate_optical_flow_from_frames(frames_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    frame_files = sorted([os.path.join(frames_dir, img) for img in os.listdir(frames_dir) if img.endswith('.jpg')])
    
    prev_frame = cv2.imread(frame_files[0])
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    for count, frame_file in enumerate(frame_files[1:]):
        frame = cv2.imread(frame_file)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        hsv = np.zeros_like(prev_frame)
        hsv[..., 1] = 255
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        flow_frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        flow_frame_path = os.path.join(output_dir, f"flow_{count:04d}.jpg")
        cv2.imwrite(flow_frame_path, flow_frame)
        
        prev_gray = gray

# Calculate optical flow for extracted frames
calculate_optical_flow_from_frames('/kaggle/working/train_frames', '/kaggle/working/train_flows')
calculate_optical_flow_from_frames('/kaggle/working/test_frames', '/kaggle/working/test_flows')
